<a href="https://colab.research.google.com/github/jisoolee11/study/blob/main/Dacon/computer_vision(test3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> DACON 제 2회 컴퓨터 비전 학습 경진대회

In [ ]:
# from google.colab import drive

# drive.mount('/content/gdrive')

In [ ]:
# !pip install torch-lr-finder

In [ ]:
cd /content/train

/content/train


In [ ]:
!unzip -qq /content/drive/MyDrive/Data/computer_vision2/dirty_mnist_2nd.zip

In [ ]:
cd ../

/content


In [ ]:
cd /content/test

/content/test


In [ ]:
!unzip -qq /content/drive/MyDrive/Data/computer_vision2/test_dirty_mnist_2nd.zip

In [ ]:
cd ../

/content


In [ ]:
import os
from typing import Tuple, Sequence, Callable
import csv
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import GradScaler
from torchsummary import summary
from torchvision import transforms
from torchvision.models import resnet101, resnet50, resnet34
# from torch_lr_finder import LRFinder

In [ ]:
class MnistDataset(Dataset):
    def __init__(
        self,
        dir: os.PathLike,
        image_ids: os.PathLike,
        transforms: Sequence[Callable]
    ) -> None:
        self.dir = dir
        self.transforms = transforms

        self.labels = {}
        with open(image_ids, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.labels[int(row[0])] = list(map(int, row[1:]))

        self.image_ids = list(self.labels.keys())

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, index: int) -> Tuple[Tensor]:
        image_id = self.image_ids[index]
        image = Image.open(
            os.path.join(
                self.dir, f'{str(image_id).zfill(5)}.png')).convert('RGB')
        target = np.array(self.labels.get(image_id)).astype(np.float32)

        if self.transforms is not None:
            image = self.transforms(image)

        return image, target

In [ ]:
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    # transforms.RandomRotation((30, 360)),
    # transforms.RandomAffine((0, 360)),
    # transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

In [ ]:
trainset = MnistDataset('/content/train', '/content/drive/MyDrive/Data/computer_vision2/dirty_mnist_2nd_answer.csv', transforms_train)
testset = MnistDataset('/content/test', '/content/drive/MyDrive/Data/computer_vision2/sample_submission.csv', transforms_test)

train_loader = DataLoader(trainset, batch_size=32, num_workers=0)
test_loader = DataLoader(testset, batch_size=4, num_workers=0)

In [ ]:
class MnistModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.resnet = resnet101(pretrained=True)
        self.classifier = nn.Linear(1000, 26)

        nn.init.xavier_normal_(self.classifier.weight)

    def forward(self, x):
        x = self.resnet(x)
        x = self.classifier(x)

        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = MnistModel().to(device)
# print(summary(model,(3, 256, 256)))

cuda


Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


In [ ]:
# optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
# criterion = nn.MultiLabelSoftMarginLoss()
# scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=2, mode='min')
# lr_finder = LRFinder(model, optimizer, criterion, device='cuda')
# lr_finder.range_test(train_loader, end_lr=10, num_iter=100)
# lr_finder.plot()
# lr_finder.reset()

In [ ]:
# optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
# criterion = nn.MultiLabelSoftMarginLoss()
# scheduler = lr_scheduler.ReduceLROnPlateau(optimizer,threshold=1,patience=1,mode='min')
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
criterion = nn.MultiLabelSoftMarginLoss()

num_epochs = 20
scaler = GradScaler()

model.train()

for epoch in range(num_epochs):
    for i, (images, targets) in enumerate(train_loader):
        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        outputs = model(images)
        loss = criterion(outputs, targets)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # loss.backward()
        # optimizer.step()

        # scheduler.step(loss)

        if (i+1) % 500 == 0:
            outputs = outputs > 0.0
            acc = (outputs == targets).float().mean()
            print(f'{epoch}: {loss.item():.5f}, {acc.item():.5f}')

0: 0.69108, 0.54567
0: 0.67868, 0.55889
0: 0.64972, 0.61058
1: 0.62954, 0.63341
1: 0.58235, 0.68630
1: 0.58739, 0.66827
2: 0.54081, 0.73798
2: 0.50891, 0.75120
2: 0.52619, 0.72596
3: 0.50885, 0.75000
3: 0.45292, 0.78846
3: 0.48521, 0.76322
4: 0.44977, 0.80409
4: 0.39530, 0.83173
4: 0.43092, 0.79688
5: 0.41141, 0.81370
5: 0.35060, 0.85457
5: 0.39952, 0.83293
6: 0.39011, 0.82572
6: 0.33258, 0.85337
6: 0.38463, 0.82452
7: 0.36137, 0.85096
7: 0.30504, 0.87260
7: 0.35217, 0.84375
8: 0.32524, 0.86298
8: 0.30435, 0.88462
8: 0.33555, 0.85697
9: 0.30115, 0.88101
9: 0.26443, 0.88462
9: 0.32792, 0.87500
10: 0.26893, 0.89303
10: 0.26768, 0.88582
10: 0.25909, 0.89543
11: 0.28876, 0.88341
11: 0.25243, 0.89784
11: 0.25593, 0.89303
12: 0.24994, 0.90505
12: 0.22444, 0.91346
12: 0.23802, 0.90505
13: 0.24531, 0.90625
13: 0.21988, 0.91226
13: 0.21927, 0.91466
14: 0.25773, 0.89904
14: 0.17442, 0.93750
14: 0.23262, 0.90144
15: 0.23302, 0.90505
15: 0.19585, 0.91947
15: 0.20840, 0.91707
16: 0.21102, 0.90986
1

In [ ]:
# lr_finder = LRFinder(model, optimizer, criterion, device='cuda')
# lr_finder.range_test(train_loader, end_lr=10, num_iter=100)
# lr_finder.plot()
# lr_finder.reset()

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/Data/computer_vision2/sample_submission.csv')

model.eval()
batch_size = test_loader.batch_size
batch_index = 0
for i, (images, targets) in enumerate(test_loader):
    images = images.to(device)
    targets = targets.to(device)
    outputs = model(images)
    outputs = outputs > 0.0
    batch_index = i * batch_size
    submit.iloc[batch_index:batch_index+batch_size, 1:] = \
        outputs.long().squeeze(0).detach().cpu().numpy()

submit.to_csv('/content/drive/MyDrive/Data/computer_vision2/submit38.csv', index=False)
